In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from optiver_features import *

In [5]:
df_train = pd.read_csv('train_with_features_NO_ST.csv')
#df_train = pd.read_csv('train_10m_with_embs.csv')
#df_train = generate_train_df(False, False)

In [4]:
#df_train.to_csv('train_with_features_NO_ST.csv', index=False)

NameError: name 'df_train' is not defined

## TODO: fill with 0 or median depending on the column

In [6]:
np.random.seed(42)
time_ids = df_train.time_id.unique()
splits = np.random.randint(0,5, len(time_ids))
split_map = dict(zip(time_ids, splits))
df_train['split'] = df_train.time_id.map(split_map)

In [7]:
#df_train = df_train.drop(['stock_id', 'time_id', 'row_id'], axis=1)
df_train = df_train.drop(['row_id'], axis=1)

In [8]:
cont_nn,cat_nn = cont_cat_split(df_train, max_card=9000, dep_var='target')

In [9]:
cat_nn.remove('split')

In [11]:
df_train = df_train.fillna(0)

In [12]:
procs_nn = [Categorify, Normalize]
df_train['is_valid'] = df_train.split == 1
#splits = ColSplitter()(df_train)
splits = RandomSplitter()(df_train)
#splits=[list(range(len(df_train))), []]
to_nn = TabularPandas(df_train, procs_nn, cat_nn, cont_nn,
                      splits=splits, 
                       
                      y_names='target')

dls = to_nn.dataloaders(1024)

In [53]:
def rmspe(preds, targs):
    x = (targs-preds)/targs
    return (x**2).mean().sqrt()

config={'lin_first':True, 'ps':[.2,.1,0], 'embed_p':0, }
learn = tabular_learner(dls, y_range=(0,.1), layers=[1000,400,200], emb_szs={'stock_id':16, 'time_id':64}, 
                        n_out=1, loss_func = rmspe, metrics=AccumMetric(rmspe), config=config,wd=0)

In [49]:
time_emb = torch.load('timeemb64.pt')

In [50]:
learn.model.embeds[1].weight.data[:,:]=time_emb

In [54]:
learn.model.embeds[1].requires_grad_(False)

Embedding(3831, 64)

In [55]:
learn.fit_one_cycle(70, 5e-3)

epoch,train_loss,valid_loss,rmspe,time
0,3.093771,2.380155,2.385258,00:03
1,1.072007,0.935532,0.941317,00:03
2,0.357378,0.293815,0.294009,00:03
3,0.271409,0.289446,0.289659,00:03
4,0.261436,0.244425,0.244615,00:03
5,0.252717,0.285154,0.285348,00:03
6,0.247561,0.242977,0.243172,00:03
7,0.254104,0.254810,0.254974,00:03
8,0.248985,0.236302,0.236531,00:03
9,0.250898,0.255805,0.256015,00:03


In [70]:
def train_fold(split, fname):
    
    #df_train['is_valid'] = df_train.split == split
    procs_nn = [Categorify, Normalize]
    #splits = ColSplitter()(df_train)
    splits = RandomSplitter()(df_train)
    to_nn = TabularPandas(df_train, procs_nn, cat_nn, cont_nn,
                          splits=splits, y_names='target')

    dls = to_nn.dataloaders(512)
    model = TabularModel([(113,23), (3831, 162)], len(cont_nn), 1, [1000,500,200], y_range=(0,.1), embed_p=.25,ps=.25)
    model.embeds[1].weight.data[:,:] = time_embs
    learn = Learner(dls, model, loss_func=rmspe, metrics=AccumMetric(rmspe), opt_func=ranger)
    learn.model.embeds[1].requires_grad_(False)
    learn.fit_flat_cos(140, 1e-3)
    score = learn.recorder.metrics[0].value
    #score = learn.early_stopping.best
    #learn.save(fname)
    return score

In [71]:
res=[]
for splt in range(5):
    res.append(train_fold(splt, f'time_split_fold_{splt}'))
    print('fold', len(res), res[-1])
np.mean(res),res

epoch,train_loss,valid_loss,rmspe,time
0,0.973891,0.749213,0.768065,00:05
1,0.342664,0.282471,0.282839,00:05
2,0.305199,0.244510,0.244982,00:05
3,0.293327,0.235522,0.235921,00:05
4,0.291719,0.233245,0.233701,00:05
5,0.289652,0.230386,0.230831,00:05
6,0.292759,0.229288,0.229760,00:05
7,0.285566,0.226699,0.227147,00:05
8,0.283614,0.233597,0.236093,00:05
9,0.275373,0.250042,0.265228,00:05


fold 1 TensorBase(2.6030)


epoch,train_loss,valid_loss,rmspe,time
0,1.052626,0.708911,0.724602,00:05
1,0.341129,0.264859,0.265149,00:05
2,0.312639,0.245813,0.246059,00:05
3,0.300574,0.239956,0.240241,00:05
4,0.290313,0.233550,0.234040,00:05
5,0.284025,0.225970,0.227175,00:05
6,0.284439,0.224887,0.225400,00:05
7,0.279443,0.229181,0.230503,00:05
8,0.277202,0.224047,0.224343,00:05
9,0.273262,0.222404,0.222894,00:05


fold 2 TensorBase(0.9652)


epoch,train_loss,valid_loss,rmspe,time
0,0.941949,0.562120,0.577523,00:05
1,0.347600,0.274008,0.274378,00:05
2,0.298901,0.238899,0.239189,00:05
3,0.287521,0.234764,0.235025,00:05
4,0.281525,0.225604,0.225904,00:05
5,0.284951,0.223692,0.223969,00:05
6,0.280309,0.221865,0.222170,00:05
7,0.276469,0.221730,0.222055,00:05
8,0.298266,0.225866,0.226409,00:05
9,0.265680,0.312777,1.140926,00:05


...

epoch,train_loss,valid_loss,rmspe,time
0,0.979968,0.624173,0.684907,00:05
1,0.330822,0.261784,0.262158,00:05
2,0.313334,0.247854,0.248281,00:05
3,0.298901,0.233248,0.233694,00:05
4,0.294763,0.235021,0.235465,00:05
5,0.288319,0.223294,0.223934,00:05
6,0.282430,0.228493,0.229573,00:05
7,0.284252,0.224408,0.224822,00:05
8,0.284745,0.221353,0.222453,00:05
9,0.275401,0.223882,0.224640,00:05


fold 4 TensorBase(0.4042)


epoch,train_loss,valid_loss,rmspe,time
0,1.014320,0.748040,0.811860,00:05
1,0.329157,0.257770,0.258076,00:05
2,0.313409,0.250372,0.250690,00:05
3,0.288647,0.231428,0.231729,00:05
4,0.285857,0.231942,0.234406,00:05
5,0.288942,0.227607,0.227874,00:05
6,0.295011,0.236759,0.237056,00:05
7,0.280030,0.226487,0.226735,00:05
8,0.277139,0.227355,0.227643,00:05
9,0.271496,0.225465,0.226658,00:05


fold 5 TensorBase(1.4947)


(1.6012818,
 [TensorBase(2.6030),
  TensorBase(0.9652),
  TensorBase(2.5393),
  TensorBase(0.4042),
  TensorBase(1.4947)])